# TWITTER ANALYTICS - ABENDI

### OBJETIVO

Coletar de forma automática dados através da ferramenta de web scraping Selenium para a geração de dashboards no Power BI para facilitar a tomada de decisão do Marketing e Diretoria.

### Métricas coletadas

- Tweets
- Impressões do Tweet
- Visitas ao perfil
- Menções
- Novos Seguidores

### Infomações de acesso

- Usuário: USUÁRIO DA PÁGINA DO TWITTER
- Senha: SENHA DO USUÁRIO

## 1 - IMPORTAÇÕES DE PACOTES

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver.v2 as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import os
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
from datetime import date, timedelta
import os
from zipfile import ZipFile
import pandas as pd
import shutil
import pyodbc
import smtplib
import email.message

## 2 - DEFINIÇÃO DE PARÂMETROS

In [2]:
day = date.today() - timedelta(days=1)
day = str(day.strftime('%d/%m/%Y')) 

metrics = []
values = []

obj = {}

remetente = 'E-MAIL DO REMETENTE' 
senha = 'SENHA DE LIBERAÇÃO DO USO DO DO CÓGIGO COM A CONTA DO GOOGLE (REMETENTE)'

## 3 - INICIALIZAÇÃO DO NAVEGADOR

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument(r'--user-data-dir=CAMINHO DO PERFIL DO GOOGLE CHROME')
service = Service(ChromeDriverManager().install())
browser = uc.Chrome(service=service, options=options)

## 4 - ACESSANDO A PÁGINA PRINCIPAL DO TWITTER 

In [4]:
browser.get('https://analytics.twitter.com/user/NOME DA PÁGINA/home')
browser.maximize_window()
time.sleep(5)

## 5 - COLETANDO INFORMAÇÕES DA PÁGINA PRINCIPAL VIA SELENIUM

In [5]:
element = browser.find_elements(By.CLASS_NAME, 'DataPoint-label')
element1 = browser.find_elements(By.CLASS_NAME,'DataPoint-info')

if browser.find_element(By.CLASS_NAME,'DataPoint-label') == 'Seguidores':
    total_followers = browser.find_element(By.CLASS_NAME,'DataPoint-info').text
else:
    total_followers = '0'

In [6]:
metricas = []
resultados = []

for i in element:
    metricas.append(i.text)
    
for i in element1:
    resultados.append(i.text)

In [8]:
for i in range(len(metricas)):
    if metricas[i] == 'Seguidores':
        total_followers = resultados[i]
        total_followers = total_followers.rsplit()
        total_followers = total_followers[0]

In [10]:
# Definindo o caminho correto das informações na página do Twitter

xpath1_value = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[1]/div/div'
xpath1_title = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[1]/div/h3'
xpath2_value = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div'
xpath2_title = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[2]/div/h3'
xpath3_value = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[3]/div/div'
xpath3_title = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[3]/div/h3'
xpath4_value = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[4]/div/div'
xpath4_title = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[4]/div/h3'
xpath5_value = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[5]/div/div'
xpath5_title = '/html/body/div[3]/div/div[2]/div[1]/div[2]/div[2]/div/div/div/div[5]/div/h3'

full_info = {xpath1_title:xpath1_value, xpath2_title:xpath2_value, xpath3_title:xpath3_value, xpath4_title:xpath4_value, 
             xpath5_title:xpath5_value}

In [ ]:
# Coletando os dados

for key, value in full_info.items():
    try:
        print(browser.find_element(By.XPATH,key).text + ' = ' + browser.find_element(By.XPATH,value).text)
        metric = browser.find_element(By.XPATH,key).text
        total = browser.find_element(By.XPATH,value).text
        metrics.append(metric)
        values.append(total)
    except:
        pass

In [12]:
# Concatenando os dados coletados em um dicionário

obj = dict(zip(metrics, values))

In [14]:
# Inserindo dados no Dicionário criado

obj['Data'] = day
obj['Total Seguidores'] = total_followers

## 6 - CRIANDO O DATAFRAME E SALVANDO OS DADOS COLETADOS

In [16]:
# Criando um dataframe padrão

default = pd.DataFrame([{'Tweets':'0','Impressões do Tweet':'0','Visitas ao perfil':'0',
                        'Novos seguidores':'0','Data':day,'Total Seguidores':'0','Menções':'0'}])

In [17]:
# Montando o Dataframe

dataframe = pd.DataFrame(data=obj, index=[0])

In [18]:
# Unindo o Dataframe padrão com o Dataframe gerado na captura dos dados

df = pd.merge(left=dataframe, right=default, how='outer')

In [19]:
# Manipulando os dados

df = df.drop(axis=0, index=1)

df = df.fillna('0')

In [21]:
# Colocando as colunas do Dataframe na ordem em que aparecem no banco

df = df[['Data','Total Seguidores','Tweets','Impressões do Tweet','Visitas ao perfil','Menções','Novos seguidores']]

In [23]:
# Salvando o dataframe para um arquivo CSV

df.to_csv(r'CAMINHO PARA SALVAR O RELATÓRIO GERADO\tb_twitter_main.csv', index=False, encoding='utf-8')

## 7 - BANCO DE DADOS

In [26]:
# Criando as variáveis

tweets = df['Tweets'][0]
impressoes = df['Impressões do Tweet'][0]
visitas = df['Visitas ao perfil'][0]
mencoes = df['Menções'][0]
novos = df['Novos seguidores'][0]

In [28]:
# Criando os dados para a conexão com o banco de dados PostgreSQL no servidor ABEDS01 (192.168.0.15)
dados_conexao = ('Driver=PostgreSQL Unicode(x64);Server=192.168.0.15;Database=mkt;UID=postgres;PWD=@bendi321')

# Criando a conexão com o banco de dados
conexao = pyodbc.connect(dados_conexao)

# Criando o cursor
cursor = conexao.cursor()

In [29]:
# Inserção dos registros coletados na tabela tb_twitter_main
cursor.execute(f"""
INSERT INTO public.tb_twitter_main(tweets,tweetviews,profileviews,mentions,newfollowers,date,followers)
VALUES('{tweets}','{impressoes}','{visitas}','{mencoes}','{novos}','{day}','{total_followers}');
""")

In [30]:
# Efetivando a inserção dos registros
cursor.commit()

In [31]:
# Fechando a conexcão com o Banco de Dados após a inserção dos registros
cursor.close()
conexao.close()

In [32]:
#browser.close()
print('Programa executado com sucesso.')

Programa executado com sucesso.


## 8 - NOTIFICANDO O RESPONSÁVEL

### INFORMAÇÕES GERAIS PARA ENVIO DE E-MAIL

    1 - Acesse a conta do Gmail que será utilizada para o envio do e-mail
    2 - Clique em "Gerenciar conta do Google" no menu do perfil
    3 - Clique em "Segurança"
    4 - Na opção "Como fazer login no Google" clique em "Senhas de App"
    5 - Clique em "Selecionar App" e escolha a opção "Outro - Nome Personalizado"
    6 - Defina um nome para o App e clique no botão GERAR
    7 - A senha que será utilizada para autenticação será exibida. Utilize esta senha na variável senha

In [ ]:
corpo_email = f"""
<p><b>***** Informativo *****</b></p>
<p>Rotina de carga de dados do Twitter realizada com sucesso no dia {day}.</p>
"""

msg = email.message.Message()
msg['Subject'] = "Carga de dados Twitter"
msg['From'] = remetente
msg['To'] = 'E-MAIL DO DESTINATÁRIO'
password = senha 
msg.add_header('Content-Type', 'text/html')
msg.set_payload(corpo_email )

s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()

s.login(msg['From'], password)
s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))

print('Email enviado')